In [1]:
import pandas as pd
from statistics import mean
from nltk.tokenize import word_tokenize

In [17]:
import numpy as np

In [2]:
#imports the data
# roughDat= pd.read_csv("PlayDat.csv")
roughDat= pd.read_csv("../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean.csv")

In [3]:
#removes unnecessary columns.
# roughDat = roughDat.drop(columns=['Unnamed: 0', 'articleHeadline', 'claimId',
#        'articleId', 'articleHeadlineStance'])

In [4]:
# import os
# os.environ["CORENLP_HOME"] = r'D:\\Downloads\\stanford-corenlp-full-2018-10-05\\stanford-corenlp-full-2018-10-05'

In [4]:
#import re 
mylist=roughDat["articleHeadline"].tolist()
#myshortlist = [re.sub(r'[^\w\s\']','',s) for s in mylist] for removing punctuation. 

In [5]:
from stanfordnlp.server import CoreNLPClient
rootIndex =[]
rootDistances = []
with CoreNLPClient(annotators=['depparse'], timeout=60000, memory='16G') as client:
    # submit the request to the server
    for sentences in mylist:

        ann = client.annotate(sentences)

        # get the first sentence
        sentence = ann.sentence[0]
        # get the dependency parse of the first sentence
        dependency_parse = sentence.basicDependencies
        root=int(dependency_parse.root[0])
        rootIndex.append(root)
        neg_Pos=None
        for i in dependency_parse.edge:
            if i.dep=="neg":
                neg_Pos=int(i.target)
        if neg_Pos!=None:
            rootDist=abs(root - neg_Pos)
        else:
            rootDist=0
        rootDistances.append(rootDist)        

Starting server with command: java -Xmx16G -cp /Users/nikilsaldanaha/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-d3a2f1a3dd49413a.props -preload depparse


In [10]:
refuting =['fake',
           'fraud',
           'hoax',
           'false',
           'deny', 
           'denies',
           'refute',
           'not',
           'despite',
           'nope',
           'doubt', 
           'doubts',
           'bogus',
           'debunk',
           'pranks',
           'prank',
           'retract']
hedging = [
        'alleged', 'allegedly',
        'apparently',
        'appear', 'appears',
        'claim', 'claims',
        'could',
        'evidently',
        'largely',
        'likely',
        'mainly',
        'may', 'maybe', 'might',
        'mostly',
        'perhaps',
        'presumably',
        'probably',
        'purported', 'purportedly',
        'reported', 'reportedly',
        'rumor', 'rumour', 'rumors', 'rumours', 'rumored', 'rumoured',
        'says',
        'seem',
        'somewhat',
        'supposedly',
        'unconfirmed']

In [11]:
mytokenizedlist =[word_tokenize(i) for i in mylist]

In [12]:
hedged_RDist = []
k = 0
for i in mytokenizedlist:
    Distances = []
    for j in hedging:
        if j in i:
            Dist = abs(rootIndex[k]-i.index(j))
            Distances.append(Dist)
        else:
            Distances.append(0)
    k+=1
    Distances = [i for i in Distances if i != 0]
    if len(Distances)==0:
        Distances.append(0)
    hedged_RDist.append(mean(Distances))

In [14]:
refute_RDist =[]
k=0
for i in mytokenizedlist:
    Distances=[]
    for j in refuting:
        if j in i:
            Dist=(abs(rootIndex[k]-i.index(j)))
            Distances.append(Dist)
        else:
            Distances.append(0)
    k+=1
    Distances = [i for i in Distances if i != 0]
    if len(Distances)==0:
        Distances.append(0)
    refute_RDist.append(mean(Distances))

In [22]:
hedged_RDist = np.array(hedged_RDist)

In [23]:
refute_RDist = np.array(refute_RDist)

In [24]:
np.savetxt("../data/processed/features/hedged_rdist.csv", hedged_RDist, delimiter=",")
np.savetxt("../data/processed/features/refute_rdist.csv", refute_RDist, delimiter=",")